In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import Binarizer
import re

# 1. Import Pandas and read in the CSV File

# 2. Check to see how the data looks like
Hint: You need a Pandas function for this

Your solution:

In [2]:
df=pd.read_csv('train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Data Cleanup Exercises
### Use the appropriate pandas function or method to perform the tasks below

#### 1. Replace male/female with boolean values
Boolean values are binary values. You can use 1/0 or True/False.

To convert the male/female entires in df.Sex to boolean types

In [3]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,C


#### 2. Fill any missing age values with the average age of the passengers

I plan to deal with the misaing numbers for the age using the titles of the passengers.
First I will make a new feature to extract the titles frol the name columns, then normalize and map them to a new mapping system I will create.

In [4]:
df['Title'] = df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) #This will create a new feature.

In [5]:
# normalizing the titles with a new title registry
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
# map the normalized titles to the new titles 
df.Title = df.Title.map(normalized_titles)
# view value counts for the normalized titles
print(df.Title.value_counts())

Mr         517
Miss       184
Mrs        127
Master      40
Officer     18
Royalty      5
Name: Title, dtype: int64


Now I will group the data by their Sex, PClass and Titles, then I will find their mean age by this grouping.

In [6]:
# group by Sex, Pclass, and Title 
grouped = df.groupby(['Sex','Pclass', 'Title'])  
# view the median Age by the grouped features 
grouped.Age.mean()

Sex  Pclass  Title  
0    1       Miss       29.744681
             Mrs        40.400000
             Officer    49.000000
             Royalty    40.500000
     2       Miss       22.390625
             Mrs        33.547619
     3       Miss       16.123188
             Mrs        33.515152
1    1       Master      5.306667
             Mr         41.580460
             Officer    51.125000
             Royalty    42.333333
     2       Master      2.258889
             Mr         32.768293
             Officer    42.000000
     3       Master      5.350833
             Mr         28.724891
Name: Age, dtype: float64

Then I will apply the grouped mean value on the null values in the Age column

In [7]:
# apply the grouped mean value on the Age null values 
df.Age = grouped.Age.apply(lambda x: x.fillna(x.mean()))

Now this takes effect on the data frame

In [8]:
df.Age

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    16.123188
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

#### 3. Use a Regex expression on the Ticket column to get just the ticket numbers attached to the ticket
A/5 21171 should now be represented as 21171

In [9]:
extr = df['Ticket'].str.extract(r'(\d{4})', expand=True)
extr.head(11)

,0
0,2117
1,1759
2,3101
3,1138
4,3734
5,3308
6,1746
7,3499
8,3477
9,2377


Another way of doing exactly the same thing is described by the code below.

extr = df['Ticket'].str.extract(pat='(\d[0-9]..)', expand=True)
extr.head(11)

Only numbers with len=4 are what can be displayed. So as not to convert other ticket numbers with less than five numbers to null values

In [14]:
df.Ticket=extr
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,2117,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,1759,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,3101,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,1138,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,3734,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",1,28.724891,0,0,3308,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,1746,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,3499,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,3477,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,2377,30.0708,NaN,C,Mrs


#### 4. Use the Sklearn Binarizer on the Fare Column
Any value less than 40 should belong to the 0 class, and values greater than 40 should belong to the 1 class. Note that sklearn has been imported from the beginning of the code.

In [10]:
fare=np.array(df.Fare)
fare=fare.reshape(1, -1) #This reshapes the column as a single array
binarizer=Binarizer(40) #Binarizes values less than 40=0, while values greater than 40=1
binarizer.fit_transform(fare) #This does the transformation

array([[0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 

#### 5. Extract the titles from the names and add it to the dataframe as a new column
Titles in the name column are Mr., Mrs., Miss., etc. Extract the title from the name and add it into a new column. The column should only include Mr, Mrs, Miss, etc.

Hints: You may need to separate the work into several lines of code. You may need to string together several functions like apply(), split(), lambda. Check the python documentation to learn more about each function.

In [11]:
df['Title'] = df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) #This will create a new feature.

In [12]:
# normalizing the titles with a new title registry
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
# map the normalized titles to the new titles 
df.Title = df.Title.map(normalized_titles)
# view value counts for the normalized titles
print(df.Title.value_counts())

Mr         517
Miss       184
Mrs        127
Master      40
Officer     18
Royalty      5
Name: Title, dtype: int64


Viewing the top 50 rows of the new dataset which now carries a new column 'Title'

In [15]:
df.head(50)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,2117,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,1759,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,3101,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,1138,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,3734,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",1,28.724891,0,0,3308,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,1746,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,3499,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,3477,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,2377,30.0708,NaN,C,Mrs
